In [1]:
!pip install cvxpy
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import cvxpy as cp
from scipy.optimize import linprog


## Revenue Model

#### B777-300ER

In [2]:
Eco_p, Eco_d= 635,180
Flex_p, Flex_d= 765,120
Prime_p, Prime_d=920,60
Business_p, Business_d=2300,49
capacity = 349

Eco = cp.Variable(integer=True)
Flex = cp.Variable(integer=True)
Prime = cp.Variable(integer=True)
Business = cp.Variable(integer=True)

objective_thy = cp.Maximize(
    Eco_p * Eco + Flex_p * Flex + Prime_p * Prime + Business_p * Business
)

# Constraints
constraints_thy = [
    Eco <= Eco_d,
    Flex <= Flex_d,
    Prime <= Prime_d,
    Business <= Business_d,
    Eco + Flex + Prime <= 300,
    Eco + Flex + Prime + Business <= capacity,
    Eco >= 0, Flex >= 0, Prime >= 0, Business >= 0
]

prob = cp.Problem(objective_thy, constraints_thy)
prob.solve(solver=cp.GLPK_MI)

print("Status:", prob.status)
print("Optimal revenue:", prob.value)
print("Eco seats:", Eco.value)
print("Flex seats:", Flex.value)
print("Prime seats:", Prime.value)
print("Business seats:", Business.value)
print("Total seats sold:", Eco.value + Flex.value + Prime.value + Business.value)

Status: optimal
Optimal revenue: 335900.0
Eco seats: 120.0
Flex seats: 120.0
Prime seats: 60.0
Business seats: 49.0
Total seats sold: 349.0


#### B787-9

In [3]:
Eco_p, Eco_d= 635,180
Flex_p, Flex_d= 765,120
Prime_p, Prime_d=920,60
Business_p, Business_d=2300,30
capacity = 300

Eco = cp.Variable(integer=True)
Flex = cp.Variable(integer=True)
Prime = cp.Variable(integer=True)
Business = cp.Variable(integer=True)

objective_thy = cp.Maximize(
    Eco_p * Eco + Flex_p * Flex + Prime_p * Prime + Business_p * Business
)

# Constraints
constraints_thy = [
    Eco <= Eco_d,
    Flex <= Flex_d,
    Prime <= Prime_d,
    Business <= Business_d,
    Eco + Flex + Prime <= 270,
    Eco + Flex + Prime + Business <= capacity,
    Eco >= 0, Flex >= 0, Prime >= 0, Business >= 0
]

prob = cp.Problem(objective_thy, constraints_thy)
prob.solve(solver=cp.GLPK_MI)

print("Status:", prob.status)
print("Optimal revenue:", prob.value)
print("Eco seats:", Eco.value)
print("Flex seats:", Flex.value)
print("Prime seats:", Prime.value)
print("Business seats:", Business.value)
print("Total seats sold:", Eco.value + Flex.value + Prime.value + Business.value)

Status: optimal
Optimal revenue: 273150.0
Eco seats: 90.0
Flex seats: 120.0
Prime seats: 60.0
Business seats: 30.0
Total seats sold: 300.0


## Fleet Model

In [25]:
flights = pd.DataFrame([
    {"fid":"TK75",  "orig":"IST", "dest":"YVR", "dist_mi":5973.9},
    {"fid":"TK76",  "orig":"YVR", "dest":"IST", "dist_mi":5973.9},

    {"fid":"TK001", "orig":"IST", "dest":"JFK", "dist_mi":5000.0}, 
    {"fid":"TK002", "orig":"JFK", "dest":"IST", "dist_mi":5000.0},

    {"fid":"TK193", "orig":"IST", "dest":"LHR", "dist_mi":1550.0},
    {"fid":"TK194", "orig":"LHR", "dest":"IST", "dist_mi":1550.0},
])
# should we derive hours later using cruising speed? -> could do that


fleet = pd.DataFrame([
    {
        "type":"B777-300ER", "seats":349, "fuel_kg_hr":7500,
        "maint_usd_hr":2200,
        "pilot_usd_hr":1800,
        "cabin_usd_hr":1200,
        "N":3, "range_mi":7600
    },
    {
        "type":"B787-9", "seats":300, "fuel_kg_hr":5600,
        "maint_usd_hr":2000,
        "pilot_usd_hr":1700,
        "cabin_usd_hr":1100,
        "N":2, "range_mi":7600
    },
])

FUEL_PRICE_USD_PER_KG = 0.90
CRUISE_MPH = 560

# daily limit per aircraft
H_MAX = 15.0  # 15 hours/day/aircraft

# flight hours
flights["hours"] = flights["dist_mi"] / CRUISE_MPH

# cost matrix c[i,j]
I, J = len(flights), len(fleet)
C = np.zeros((I, J))

for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        hours = f["hours"]

        fuel_cost = a["fuel_kg_hr"] * hours * FUEL_PRICE_USD_PER_KG
        maint_cost = a["maint_usd_hr"] * hours
        crew_cost = (a["pilot_usd_hr"] + a["cabin_usd_hr"]) * hours

        C[i, j] = fuel_cost + maint_cost + crew_cost

x = cp.Variable((I, J), boolean=True)

constraints = []

constraints += [cp.sum(x[i, :]) == 1 for i in range(I)]

pairs = [("TK75", "TK76"), ("TK001", "TK002"), ("TK193", "TK194")]

fid_to_idx = {fid: idx for idx, fid in enumerate(flights["fid"])}

for f1, f2 in pairs:
    i1, i2 = fid_to_idx[f1], fid_to_idx[f2]
    constraints += [x[i1, :] == x[i2, :]]  # same fleet type chosen


# Total crew-hours used by fleet type j = sum_i (hours_i * x_{i,j})
# must be <= (number of available aircraft of type j) * (max crew-hours per aircraft per day)
CREW_H_MAX = H_MAX
for j in range(J):
    crew_hours_used_j = cp.sum(cp.multiply(flights["hours"].values, x[:, j]))
    constraints += [crew_hours_used_j <= int(fleet.loc[j, "N"]) * CREW_H_MAX]

# If distance > range, x[i,j] = 0
for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        if float(f["dist_mi"]) > float(a["range_mi"]):
            constraints += [x[i, j] == 0]

objective = cp.Minimize(cp.sum(cp.multiply(C, x)))

prob = cp.Problem(objective, constraints)
prob.solve()

print("Status:", prob.status)
print("Min cost:", prob.value)

assignments = []
for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))

    assignments.append({
        "flight": f["fid"],
        "route": f"{f['orig']}-{f['dest']}",
        "assigned_fleet": fleet.loc[j_star, "type"],
        "hours": float(f["hours"]),
        "fuel_cost_usd": float(
            fleet.loc[j_star, "fuel_kg_hr"] * f["hours"] * FUEL_PRICE_USD_PER_KG
        ),
        "crew_cost_usd": float(
            (fleet.loc[j_star, "pilot_usd_hr"] + fleet.loc[j_star, "cabin_usd_hr"]) * f["hours"]
        ),
        "total_cost_usd": float(C[i, j_star])
    })

print(pd.DataFrame(assignments))

Status: optimal
Min cost: 477804.19999999995
  flight  route   assigned_fleet    hours    fuel_cost_usd  crew_cost_usd  \
0   TK75  IST-YVR       B787-9    10.667679  53765.100000   29869.500000    
1   TK76  YVR-IST       B787-9    10.667679  53765.100000   29869.500000    
2  TK001  IST-JFK   B777-300ER     8.928571  60267.857143   26785.714286    
3  TK002  JFK-IST   B777-300ER     8.928571  60267.857143   26785.714286    
4  TK193  IST-LHR       B787-9     2.767857  13950.000000    7750.000000    
5  TK194  LHR-IST       B787-9     2.767857  13950.000000    7750.000000    

   total_cost_usd  
0   104969.957143  
1   104969.957143  
2   106696.428571  
3   106696.428571  
4    27235.714286  
5    27235.714286  


#### Profit Model

In [6]:
fleet = fleet.copy()
fleet["eco_cap"] = [300, 270]
fleet["bus_cap"] = [49, 30]  
fleet["total_cap"] = fleet["eco_cap"] + fleet["bus_cap"]

I, J = len(flights), len(fleet)

prices = {"Eco": 635, "Flex": 765, "Prime": 920, "Business": 2300}
demands = {"Eco": 180, "Flex": 120, "Prime": 60, "Business": 49}

K = ["Eco", "Flex", "Prime", "Business"]

n = {k: cp.Variable(I, integer=True) for k in K}

seat_constraints = []

# nonneg + demand caps
for k in K:
    seat_constraints += [n[k] >= 0]
    seat_constraints += [n[k] <= demands[k]]

eco_cap_vec = fleet["eco_cap"].values
bus_cap_vec = fleet["bus_cap"].values
tot_cap_vec = fleet["total_cap"].values

for i in range(I):
    eco_capacity_i = eco_cap_vec @ x[i, :]
    bus_capacity_i = bus_cap_vec @ x[i, :]
    tot_capacity_i = tot_cap_vec @ x[i, :]

    seat_constraints += [
        n["Eco"][i] + n["Flex"][i] + n["Prime"][i] <= eco_capacity_i
    ]
    seat_constraints += [
        n["Business"][i] <= bus_capacity_i
    ]
    seat_constraints += [
        n["Eco"][i] + n["Flex"][i] + n["Prime"][i] + n["Business"][i] <= tot_capacity_i
    ]

constraints_all = constraints + seat_constraints

Revenue = 0
for k in K:
    Revenue += prices[k] * cp.sum(n[k])

Cost = cp.sum(cp.multiply(C, x))

rev_ub = I * (
    prices["Eco"]*demands["Eco"] +
    prices["Flex"]*demands["Flex"] +
    prices["Prime"]*demands["Prime"] +
    prices["Business"]*demands["Business"]
)

cost_ub = I * np.max(C)

Revenue_norm = Revenue / max(rev_ub, 1.0)
Cost_norm = Cost / max(cost_ub, 1.0)

lam = 0.7

objective = cp.Maximize(lam * Revenue_norm - (1 - lam) * Cost_norm)

prob = cp.Problem(objective, constraints_all)
prob.solve(solver=cp.GLPK_MI)

print("Status:", prob.status)
print("Weighted objective value:", prob.value)
print("Revenue:", Revenue.value)
print("Cost:", Cost.value)
print("Profit (Revenue - Cost):", (Revenue.value - Cost.value))

assignments = []
for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))
    assignments.append({
        "flight": f["fid"],
        "route": f"{f['orig']}-{f['dest']}",
        "assigned_fleet": fleet.loc[j_star, "type"],
        "hours": float(f["hours"]),
        "cost_usd": float(C[i, j_star]),
        "Eco": int(round(n["Eco"].value[i])),
        "Flex": int(round(n["Flex"].value[i])),
        "Prime": int(round(n["Prime"].value[i])),
        "Business": int(round(n["Business"].value[i])),
    })

print(pd.DataFrame(assignments))


Status: optimal
Weighted objective value: 0.4190462231741945
Revenue: 2015400.0
Cost: 534502.1607142857
Profit (Revenue - Cost): 1480897.8392857143
  flight    route assigned_fleet      hours       cost_usd  Eco  Flex  Prime  \
0   TK75  IST-YVR     B777-300ER  10.667679  127478.758929  120   120     60   
1   TK76  YVR-IST     B777-300ER  10.667679  127478.758929  120   120     60   
2  TK001  IST-JFK     B777-300ER   8.928571  106696.428571  120   120     60   
3  TK002  JFK-IST     B777-300ER   8.928571  106696.428571  120   120     60   
4  TK193  IST-LHR     B777-300ER   2.767857   33075.892857  120   120     60   
5  TK194  LHR-IST     B777-300ER   2.767857   33075.892857  120   120     60   

   Business  
0        49  
1        49  
2        49  
3        49  
4        49  
5        49  


In [7]:
lams = [0.0, 0.2, 0.5, 0.8, 1.0]
results = []

for lam in lams:
    objective = cp.Maximize(lam * Revenue_norm - (1 - lam) * Cost_norm)
    prob = cp.Problem(objective, constraints_all)
    prob.solve(solver=cp.GLPK_MI)

    results.append({
        "lambda": lam,
        "Revenue": Revenue.value,
        "Cost": Cost.value,
        "Profit": Revenue.value - Cost.value
    })

print(pd.DataFrame(results))


   lambda    Revenue           Cost        Profit
0     0.0        0.0  477804.200000 -4.778042e+05
1     0.2  1764400.0  477804.200000  1.286596e+06
2     0.5  1889900.0  489484.557143  1.400415e+06
3     0.8  2015400.0  534502.160714  1.480898e+06
4     1.0  2015400.0  534502.160714  1.480898e+06


[0.5, 0.8] interval lamda value almost produces the same result. 

In [8]:
import numpy as np
import pandas as pd
import cvxpy as cp

fleet = fleet.copy()
fleet["eco_cap"] = [300, 270]   # B777, B787 economy seats
fleet["bus_cap"] = [49, 30]     # business seats
fleet["total_cap"] = fleet["eco_cap"] + fleet["bus_cap"]

I, J = len(flights), len(fleet)

eco_cap_vec = fleet["eco_cap"].values.astype(float)
bus_cap_vec = fleet["bus_cap"].values.astype(float)
tot_cap_vec = fleet["total_cap"].values.astype(float)

def route_key(row):
    o, d = row["orig"], row["dest"]
    return "-".join(sorted([o, d]))

flights["route_key"] = flights.apply(route_key, axis=1)

route_params = {
    "IST-YVR": {
        "P": {"Eco": 635, "Flex": 765, "Prime": 920, "Business": 2300},
        "D": {"Eco": 180, "Flex": 120, "Prime": 60,  "Business": 49},
    },
    "IST-JFK": {
        "P": {"Eco": 520, "Flex": 660, "Prime": 820, "Business": 2100},
        "D": {"Eco": 200, "Flex": 140, "Prime": 70,  "Business": 49},
    },
    "IST-LHR": {
        "P": {"Eco": 480, "Flex": 540, "Prime": 720, "Business": 1900},
        "D": {"Eco": 220, "Flex": 160, "Prime": 90,  "Business": 4},
    },
}

K = ["Eco", "Flex", "Prime", "Business"]
I = len(flights)

P = {k: np.zeros(I) for k in K}
D = {k: np.zeros(I) for k in K}

for i, row in flights.iterrows():
    rk = row["route_key"]
    for k in K:
        P[k][i] = route_params[rk]["P"][k]
        D[k][i] = route_params[rk]["D"][k]

n = {k: cp.Variable(I, integer=True) for k in K}

seat_constraints = []

for k in K:
    seat_constraints += [n[k] >= 0]
    seat_constraints += [n[k] <= D[k]]

for i in range(I):
    eco_capacity_i = eco_cap_vec @ x[i, :]   
    bus_capacity_i = bus_cap_vec @ x[i, :]
    tot_capacity_i = tot_cap_vec @ x[i, :]

    seat_constraints += [
        n["Eco"][i] + n["Flex"][i] + n["Prime"][i] <= eco_capacity_i
    ]
    seat_constraints += [
        n["Business"][i] <= bus_capacity_i
    ]
    seat_constraints += [
        n["Eco"][i] + n["Flex"][i] + n["Prime"][i] + n["Business"][i] <= tot_capacity_i
    ]

constraints_all = constraints + seat_constraints

Revenue = 0
for k in K:
    Revenue += P[k] @ n[k]

for k in K:
    seat_constraints += [n[k] <= D[k]]


Cost = cp.sum(cp.multiply(C, x))

rev_ub = float(np.sum([np.sum(P[k] * D[k]) for k in K]))
cost_ub = float(I * np.max(C))

Revenue_norm = Revenue / max(rev_ub, 1.0)
Cost_norm = Cost / max(cost_ub, 1.0)

lam = 0.7
objective = cp.Maximize(lam * Revenue_norm - (1 - lam) * Cost_norm)

prob = cp.Problem(objective, constraints_all)
prob.solve(solver=cp.GLPK_MI)

print("Status:", prob.status)
print("Weighted objective value:", prob.value)
print("Revenue:", Revenue.value)
print("Cost:", Cost.value)
print("Profit (Revenue - Cost):", Revenue.value - Cost.value)

assignments = []
for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))

    assignments.append({
        "flight": f["fid"],
        "route": f"{f['orig']}-{f['dest']}",
        "assigned_fleet": fleet.loc[j_star, "type"],
        "hours": float(f["hours"]),
        "cost_usd": float(C[i, j_star]),
        "Eco": int(round(n["Eco"].value[i])),
        "Flex": int(round(n["Flex"].value[i])),
        "Prime": int(round(n["Prime"].value[i])),
        "Business": int(round(n["Business"].value[i])),
        "EcoCap": int(fleet.loc[j_star, "eco_cap"]),
        "BusCap": int(fleet.loc[j_star, "bus_cap"]),
        "TotCap": int(fleet.loc[j_star, "total_cap"]),
    })

print(pd.DataFrame(assignments))


Status: optimal
Weighted objective value: 0.3659166288034892
Revenue: 1636400.0
Cost: 534502.1607142857
Profit (Revenue - Cost): 1101897.8392857143
  flight    route assigned_fleet      hours       cost_usd  Eco  Flex  Prime  \
0   TK75  IST-YVR     B777-300ER  10.667679  127478.758929  120   120     60   
1   TK76  YVR-IST     B777-300ER  10.667679  127478.758929  120   120     60   
2  TK001  IST-JFK     B777-300ER   8.928571  106696.428571   90   140     70   
3  TK002  JFK-IST     B777-300ER   8.928571  106696.428571   90   140     70   
4  TK193  IST-LHR     B777-300ER   2.767857   33075.892857   50   160     90   
5  TK194  LHR-IST     B777-300ER   2.767857   33075.892857   50   160     90   

   Business  EcoCap  BusCap  TotCap  
0        49     300      49     349  
1        49     300      49     349  
2        49     300      49     349  
3        49     300      49     349  
4         4     300      49     349  
5         4     300      49     349  


In [9]:
lams = [0.0, 0.2, 0.5, 0.8, 1.0]
results = []

for lam in lams:
    objective = cp.Maximize(lam * Revenue_norm - (1 - lam) * Cost_norm)
    prob = cp.Problem(objective, constraints_all)
    prob.solve(solver=cp.GLPK_MI)

    results.append({
        "lambda": lam,
        "Revenue": Revenue.value,
        "Cost": Cost.value,
        "Profit": Revenue.value - Cost.value
    })

print(pd.DataFrame(results))


   lambda    Revenue           Cost        Profit
0     0.0        0.0  477804.200000 -4.778042e+05
1     0.2  1482100.0  477804.200000  1.004296e+06
2     0.5  1607600.0  522821.803571  1.084778e+06
3     0.8  1636400.0  534502.160714  1.101898e+06
4     1.0  1636400.0  534502.160714  1.101898e+06


## Full model

In [23]:
flights = pd.DataFrame([
    {
        "fid": "TK75", "orig": "IST", "dest": "YVR", "dist_mi": 5973.9,
        "Eco_p": 700, "Flex_p": 850, "Prime_p": 1000, "Bus_p": 2800,
        "Eco_d": 150, "Flex_d": 100, "Prime_d": 60, "Bus_d": 55
    },
    {
        "fid": "TK76", "orig": "YVR", "dest": "IST", "dist_mi": 5973.9,
        "Eco_p": 700, "Flex_p": 850, "Prime_p": 1000, "Bus_p": 2800,
        "Eco_d": 160, "Flex_d": 110, "Prime_d": 50, "Bus_d": 58
    },
    {
        "fid": "TK001", "orig": "IST", "dest": "JFK", "dist_mi": 5000.0,
        "Eco_p": 650, "Flex_p": 780, "Prime_p": 950, "Bus_p": 2500,
        "Eco_d": 200, "Flex_d": 120, "Prime_d": 70, "Bus_d": 50
    },
    {
        "fid": "TK002", "orig": "JFK", "dest": "IST", "dist_mi": 5000.0,
        "Eco_p": 650, "Flex_p": 780, "Prime_p": 950, "Bus_p": 2500,
        "Eco_d": 220, "Flex_d": 130, "Prime_d": 65, "Bus_d": 55
    },
    {
        "fid": "TK193", "orig": "IST", "dest": "LHR", "dist_mi": 1550.0,
        "Eco_p": 300, "Flex_p": 400, "Prime_p": 550, "Bus_p": 1200,
        "Eco_d": 280, "Flex_d": 150, "Prime_d": 90, "Bus_d": 55
    },
    {
        "fid": "TK194", "orig": "LHR", "dest": "IST", "dist_mi": 1550.0,
        "Eco_p": 300, "Flex_p": 400, "Prime_p": 550, "Bus_p": 1200,
        "Eco_d": 260, "Flex_d": 140, "Prime_d": 85, "Bus_d": 58
    },
    {
        "fid": "TK17", "orig": "IST", "dest": "YYZ", "dist_mi": 5050.0,
        "Eco_p": 600, "Flex_p": 800, "Prime_p": 950, "Bus_p": 2200,
        "Eco_d": 280, "Flex_d": 150, "Prime_d": 90, "Bus_d": 55
    },
    {
        "fid": "TK18", "orig": "YYZ", "dest": "IST", "dist_mi": 5050.0,
        "Eco_p": 600, "Flex_p": 800, "Prime_p": 950, "Bus_p": 2200,
        "Eco_d": 260, "Flex_d": 140, "Prime_d": 85, "Bus_d": 58
    },
    {
        "fid": "TK1821", "orig": "IST", "dest": "CDG", "dist_mi": 1050.0,
        "Eco_p": 100, "Flex_p": 150, "Prime_p": 250, "Bus_p": 1000,
        "Eco_d": 280, "Flex_d": 150, "Prime_d": 90, "Bus_d": 55
    },
    {
        "fid": "TK1830", "orig": "CDG", "dest": "IST", "dist_mi": 1050.0,
        "Eco_p": 100, "Flex_p": 150, "Prime_p": 250, "Bus_p": 1000,
        "Eco_d": 260, "Flex_d": 140, "Prime_d": 85, "Bus_d": 58
    },
    {
        "fid": "TK1523", "orig": "IST", "dest": "DUS", "dist_mi": 1250.0,
        "Eco_p": 100, "Flex_p": 150, "Prime_p": 250, "Bus_p": 1000,
        "Eco_d": 280, "Flex_d": 150, "Prime_d": 90, "Bus_d": 55
    },
    {
        "fid": "TK1530", "orig": "DUS", "dest": "IST", "dist_mi": 1250.0,
        "Eco_p": 100, "Flex_p": 150, "Prime_p": 250, "Bus_p": 1000,
        "Eco_d": 260, "Flex_d": 140, "Prime_d": 85, "Bus_d": 58
    },
    {
        "fid": "TK1951", "orig": "IST", "dest": "AMS", "dist_mi": 950.0,
        "Eco_p": 200, "Flex_p": 350, "Prime_p": 450, "Bus_p": 1300,
        "Eco_d": 280, "Flex_d": 150, "Prime_d": 90, "Bus_d": 55
    },
    {
        "fid": "TK1952", "orig": "AMS", "dest": "IST", "dist_mi": 950.0,
        "Eco_p": 200, "Flex_p": 350, "Prime_p": 450, "Bus_p": 1300,
        "Eco_d": 260, "Flex_d": 140, "Prime_d": 85, "Bus_d": 58
    },
])


fleet = pd.DataFrame([
    {
        "type": "B777-300ER", "seats": 349,
        "fuel_kg_hr": 7500, "maint_usd_hr": 2200,
        "pilot_usd_hr": 1800, "cabin_usd_hr": 1200,
        "N": 3, "range_mi": 7600,
        "biz_seats": 49, "econ_seats": 300
    },
    {
        "type": "B787-9", "seats": 290,
        "fuel_kg_hr": 5600, "maint_usd_hr": 2000,
        "pilot_usd_hr": 1700, "cabin_usd_hr": 1100,
        "N": 3, "range_mi": 7600,
        "biz_seats": 30, "econ_seats": 270
    },
])

FUEL_PRICE_USD_PER_KG = 0.90
CRUISE_MPH = 560
H_MAX = 15.0 

flights["hours"] = flights["dist_mi"] / CRUISE_MPH

Eco_p = flights["Eco_p"].values
Flex_p = flights["Flex_p"].values
Prime_p = flights["Prime_p"].values
Bus_p = flights["Bus_p"].values

Eco_d = flights["Eco_d"].values
Flex_d = flights["Flex_d"].values
Prime_d = flights["Prime_d"].values
Bus_d = flights["Bus_d"].values

biz_cap = fleet["biz_seats"].values.astype(float)
econ_cap = fleet["econ_seats"].values.astype(float)

I, J = len(flights), len(fleet)
C = np.zeros((I, J))

for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        hours = f["hours"]
        fuel_cost = a["fuel_kg_hr"] * hours * FUEL_PRICE_USD_PER_KG
        maint_cost = a["maint_usd_hr"] * hours
        crew_cost = (a["pilot_usd_hr"] + a["cabin_usd_hr"]) * hours
        C[i, j] = fuel_cost + maint_cost + crew_cost

x = cp.Variable((I, J), boolean=True)  # fleet assignment
Eco = cp.Variable(I, integer=True)     # economy seats sold
Flex = cp.Variable(I, integer=True)    # flex seats sold
Prime = cp.Variable(I, integer=True)   # prime seats sold
Bus = cp.Variable(I, integer=True)     # business seats sold


total_revenue = cp.sum(
    cp.multiply(Eco_p, Eco) +
    cp.multiply(Flex_p, Flex) +
    cp.multiply(Prime_p, Prime) +
    cp.multiply(Bus_p, Bus)
)

total_cost = cp.sum(cp.multiply(C, x))
Lambda = 0.6
objective = cp.Maximize(Lambda*total_revenue - (1-Lambda)*total_cost)

constraints = []

# (A) Each flight gets exactly one fleet type
constraints += [cp.sum(x[i, :]) == 1 for i in range(I)]

# (B) Round-trip pairing: same aircraft for outbound/return
pairs = [("TK75", "TK76"), ("TK001", "TK002"), ("TK193", "TK194"),("TK17", "TK18"),("TK1821", "TK1830"),("TK1523", "TK1530"),("TK1951", "TK1952")]
fid_to_idx = {fid: idx for idx, fid in enumerate(flights["fid"])}

for f1, f2 in pairs:
    i1, i2 = fid_to_idx[f1], fid_to_idx[f2]
    constraints += [x[i1, :] == x[i2, :]]

# (C) Crew-hours availability
for j in range(J):
    crew_hours_used = cp.sum(cp.multiply(flights["hours"].values, x[:, j]))
    constraints += [crew_hours_used <= int(fleet.loc[j, "N"]) * H_MAX]

# (D) Range feasibility
for i, f in flights.iterrows():
    for j, a in fleet.iterrows():
        if float(f["dist_mi"]) > float(a["range_mi"]):
            constraints += [x[i, j] == 0]

# (E) Demand constraints: seats sold <= demand (per flight!)
constraints += [
    Eco >= 0, Flex >= 0, Prime >= 0, Bus >= 0,
    Eco <= Eco_d,      # each flight has its own demand limit
    Flex <= Flex_d,
    Prime <= Prime_d,
    Bus <= Bus_d
]

# (F) Capacity constraints: seats sold <= aircraft capacity
for i in range(I):
    constraints += [
        Bus[i] <= biz_cap @ x[i, :],
        Eco[i] + Flex[i] + Prime[i] <= econ_cap @ x[i, :],
    ]

prob = cp.Problem(objective, constraints)
prob.solve()


print(f"Status: {prob.status}")
print(f"Maximum Profit: ${prob.value:,.2f}")
print(f"Total Revenue:  ${float(total_revenue.value):,.2f}")
print(f"Total Cost:     ${float(total_cost.value):,.2f}")
print()

print("FLIGHT ASSIGNMENTS:")
print("-" * 70)

for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))
    
    eco_sold = float(Eco.value[i])
    flex_sold = float(Flex.value[i])
    prime_sold = float(Prime.value[i])
    biz_sold = float(Bus.value[i])
    
    revenue_i = (Eco_p[i] * eco_sold + Flex_p[i] * flex_sold +
                 Prime_p[i] * prime_sold + Bus_p[i] * biz_sold)
    cost_i = float(C[i, j_star])
    profit_i = revenue_i - cost_i
    
    print(f"\nFlight {f['fid']} ({f['orig']}-{f['dest']})")
    print(f"  Fleet: {fleet.loc[j_star, 'type']}")
    print(f"  Seats sold: Eco={int(eco_sold)}/{int(Eco_d[i])}, "
          f"Flex={int(flex_sold)}/{int(Flex_d[i])}, "
          f"Prime={int(prime_sold)}/{int(Prime_d[i])}, "
          f"Biz={int(biz_sold)}/{int(Bus_d[i])}")
    print(f"  Revenue: ${revenue_i:,.0f}  |  Cost: ${cost_i:,.0f}  |  Profit: ${profit_i:,.0f}")

Status: optimal
Maximum Profit: $1,486,156.18
Total Revenue:  $3,027,850.00
Total Cost:     $826,384.56

FLIGHT ASSIGNMENTS:
----------------------------------------------------------------------

Flight TK75 (IST-YVR)
  Fleet: B787-9
  Seats sold: Eco=110/150, Flex=100/100, Prime=60/60, Biz=30/55
  Revenue: $306,000  |  Cost: $104,970  |  Profit: $201,030

Flight TK76 (YVR-IST)
  Fleet: B787-9
  Seats sold: Eco=110/160, Flex=110/110, Prime=50/50, Biz=30/58
  Revenue: $304,500  |  Cost: $104,970  |  Profit: $199,530

Flight TK001 (IST-JFK)
  Fleet: B777-300ER
  Seats sold: Eco=110/200, Flex=120/120, Prime=70/70, Biz=49/50
  Revenue: $354,100  |  Cost: $106,696  |  Profit: $247,404

Flight TK002 (JFK-IST)
  Fleet: B777-300ER
  Seats sold: Eco=105/220, Flex=130/130, Prime=65/65, Biz=49/55
  Revenue: $353,900  |  Cost: $106,696  |  Profit: $247,204

Flight TK193 (IST-LHR)
  Fleet: B777-300ER
  Seats sold: Eco=60/280, Flex=150/150, Prime=90/90, Biz=49/55
  Revenue: $186,300  |  Cost: $33,0

In [24]:

#Got help with visualiziation from claudeAI

# =============================================================================
# BUILD RESULTS TABLE
# =============================================================================

results = []
for i, f in flights.iterrows():
    j_star = int(np.argmax(x.value[i, :]))
    
    eco_sold = int(round(Eco.value[i]))
    flex_sold = int(round(Flex.value[i]))
    prime_sold = int(round(Prime.value[i]))
    biz_sold = int(round(Bus.value[i]))
    
    econ_total = eco_sold + flex_sold + prime_sold
    econ_cap_assigned = int(fleet.loc[j_star, "econ_seats"])
    biz_cap_assigned = int(fleet.loc[j_star, "biz_seats"])
    total_cap = int(fleet.loc[j_star, "seats"])
    total_sold = econ_total + biz_sold
    
    revenue_i = (Eco_p[i] * eco_sold + Flex_p[i] * flex_sold +
                 Prime_p[i] * prime_sold + Bus_p[i] * biz_sold)
    cost_i = C[i, j_star]
    profit_i = revenue_i - cost_i
    load_factor = (total_sold / total_cap) * 100
    
    results.append({
        "Flight": f["fid"],
        "Route": f"{f['orig']}-{f['dest']}",
        "Aircraft": fleet.loc[j_star, "type"],
        "Hours": round(f["hours"], 1),
        "Eco": f"{eco_sold}/{int(Eco_d[i])}",
        "Flex": f"{flex_sold}/{int(Flex_d[i])}",
        "Prime": f"{prime_sold}/{int(Prime_d[i])}",
        "Biz": f"{biz_sold}/{int(Bus_d[i])}",
        "Load %": f"{load_factor:.1f}%",
        "Revenue": revenue_i,
        "Cost": cost_i,
        "Profit": profit_i
    })

df_results = pd.DataFrame(results)

# =============================================================================
# PRINT FORMATTED OUTPUT
# =============================================================================

print("\n" + "=" * 90)
print(" " * 25 + "FLEET ASSIGNMENT OPTIMIZATION RESULTS")
print("=" * 90)

print(f"\n{'OPTIMIZATION STATUS':^90}")
print("-" * 90)
print(f"  Status:           {prob.status.upper()}")
print(f"  Objective Value:  ${prob.value:,.2f}")
print(f"  Lambda (λ):       {Lambda}")

print(f"\n{'FINANCIAL SUMMARY':^90}")
print("-" * 90)
total_rev = float(total_revenue.value)
total_cst = float(total_cost.value)
total_profit = total_rev - total_cst
margin = (total_profit / total_rev) * 100

print(f"  Total Revenue:    ${total_rev:>15,.2f}")
print(f"  Total Cost:       ${total_cst:>15,.2f}")
print(f"  Total Profit:     ${total_profit:>15,.2f}")
print(f"  Profit Margin:    {margin:>15.1f}%")

print(f"\n{'FLEET UTILIZATION':^90}")
print("-" * 90)
for j, a in fleet.iterrows():
    hours_used = sum(flights.loc[i, "hours"] * x.value[i, j] for i in range(I))
    hours_available = int(a["N"]) * H_MAX
    util_pct = (hours_used / hours_available) * 100 if hours_available > 0 else 0
    num_flights = sum(1 for i in range(I) if x.value[i, j] > 0.5)
    print(f"  {a['type']:12s}  │  {num_flights:2d} flights  │  "
          f"{hours_used:5.1f} / {hours_available:4.0f} hrs  │  {util_pct:5.1f}% utilized")

print(f"\n{'FLIGHT ASSIGNMENTS':^90}")
print("-" * 90)

# Format the dataframe for display
df_display = df_results.copy()
df_display["Revenue"] = df_display["Revenue"].apply(lambda x: f"${x:,.0f}")
df_display["Cost"] = df_display["Cost"].apply(lambda x: f"${x:,.0f}")
df_display["Profit"] = df_display["Profit"].apply(lambda x: f"${x:,.0f}")

# Print with nice formatting
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.colheader_justify', 'center')

print(df_display.to_string(index=False))

print("\n" + "-" * 90)
print(f"{'TOTALS':>58} │ ${total_rev:>10,.0f} │ ${total_cst:>8,.0f} │ ${total_profit:>9,.0f}")
print("=" * 90)

# =============================================================================
# ADDITIONAL: SEAT SALES BREAKDOWN TABLE
# =============================================================================

print(f"\n{'SEAT SALES BREAKDOWN':^90}")
print("-" * 90)

seat_breakdown = []
for i, f in flights.iterrows():
    eco_sold = int(round(Eco.value[i]))
    flex_sold = int(round(Flex.value[i]))
    prime_sold = int(round(Prime.value[i]))
    biz_sold = int(round(Bus.value[i]))
    
    eco_rev = Eco_p[i] * eco_sold
    flex_rev = Flex_p[i] * flex_sold
    prime_rev = Prime_p[i] * prime_sold
    biz_rev = Bus_p[i] * biz_sold
    
    seat_breakdown.append({
        "Flight": f["fid"],
        "Route": f"{f['orig']}-{f['dest']}",
        "Eco Seats": eco_sold,
        "Eco Rev": f"${eco_rev:,}",
        "Flex Seats": flex_sold,
        "Flex Rev": f"${flex_rev:,}",
        "Prime Seats": prime_sold,
        "Prime Rev": f"${prime_rev:,}",
        "Biz Seats": biz_sold,
        "Biz Rev": f"${biz_rev:,}"
    })

df_seats = pd.DataFrame(seat_breakdown)
print(df_seats.to_string(index=False))
print("=" * 90)


                         FLEET ASSIGNMENT OPTIMIZATION RESULTS

                                   OPTIMIZATION STATUS                                    
------------------------------------------------------------------------------------------
  Status:           OPTIMAL
  Objective Value:  $1,486,156.18
  Lambda (λ):       0.6

                                    FINANCIAL SUMMARY                                     
------------------------------------------------------------------------------------------
  Total Revenue:    $   3,027,850.00
  Total Cost:       $     826,384.56
  Total Profit:     $   2,201,465.44
  Profit Margin:               72.7%

                                    FLEET UTILIZATION                                     
------------------------------------------------------------------------------------------
  B777-300ER    │   8 flights  │   44.8 /   45 hrs  │   99.6% utilized
  B787-9        │   6 flights  │   29.5 /   45 hrs  │   65.7% utilized

          